In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 31.9 MB/s 
     |████████████████████████████████| 596 kB 36.8 MB/s 
     |████████████████████████████████| 895 kB 65.9 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 6.5 MB 57.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
import os 
import sys
import pandas as pd
import numpy as np

import re

from transformers import AutoTokenizer, AutoModelForCausalLM 

In [4]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
csv_test = pd.read_csv("/content/drive/MyDrive/NLP/sports_news_data - sports_news_data.csv")

In [7]:
csv_test.head(20)

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
5,"긴급 수혈된 바르사 NO.9, 1년 반 만에 떠난다… ‘EPL행 유력’",[스포탈코리아] FC 바르셀로나는 새 시즌을 앞두고 선수단 정리가 한창이다. 잉여 ...,2021-07-15
6,"[김남구의 유럽통신] 황의조, 손흥민 소속사와 손잡다… CAA Base와 계약",[스포탈코리아=파리(프랑스)] 황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 ...,2021-07-15
7,"""메시 종신은 축복!""…스폰서 5년 더 보장, 바르셀로나 함박웃음",[스포탈코리아] 리오넬 메시(34)가 FC바르셀로나에 남는다. 연봉을 절반 삭감하지...,2021-07-15
8,"[오피셜] 눈물 흘렸던 '37세 전설' 로번, 두 번째 현역 은퇴 발표",[스포탈코리아] 네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다. \n\...,2021-07-15
9,"100세' 메시팬 할아버지, 748골 수기 작성…메시도 감사 인사",[스포탈코리아] 리오넬 메시(34)는 프로 데뷔하고 748골을 터뜨렸다. 전산화 하...,2021-07-15


### 크롤링 상에서 생긴 쓸모 없는 문구 처리 

In [8]:
trimmed_data=csv_test.dropna().copy()
drop_list = []
for i in range(len(trimmed_data)):
  if type(trimmed_data["CONTENT"].iloc[i]) != type("문자열"):
    j = trimmed_data.iloc[i].name
    drop_list.append(j)
    continue

  trimmed_data["CONTENT"].iloc[i] = " ".join(trimmed_data["CONTENT"].iloc[i].split())
  rmve_bracket = "\[.*\]|\(.*\)|\<.+\>"
  trimmed_data["CONTENT"].iloc[i] = re.sub(rmve_bracket,"",trimmed_data["CONTENT"].iloc[i])
  mkd_mark = "[가-힣]{1,4}\s기자\=\s|사진=[가-힣]+"
  trimmed_data["CONTENT"].iloc[i] = re.sub(mkd_mark,"",trimmed_data["CONTENT"].iloc[i])
  trimmed_data["CONTENT"].iloc[i] = re.sub(r"[^\w\d가-힣\s+]","",trimmed_data["CONTENT"].iloc[i]).strip()
  trimmed_data["TITLE"].iloc[i] = re.sub(r"[^a-zA-Z0-9가-힣\s+]","",trimmed_data["TITLE"].iloc[i]).strip()

  if len(trimmed_data["CONTENT"].iloc[i]) < 20 :
    j = trimmed_data.iloc[i].name
    drop_list.append(j)

trimmed_data = trimmed_data.drop(index = drop_list)

In [10]:
len(trimmed_data)

7259

### 띄어쓰기 및 불용어 처리

In [ ]:
!pip install konlpy

In [32]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [ ]:
from konlpy.tag import Okt
okt = Okt()
for i in tqdm(range(len(trimmed_data))):
  temp_data = okt.morphs(trimmed_data["CONTENT"].iloc[i])
  temp_list = []
  
  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  trimmed_data["CONTENT"].iloc[i] = " ".join(temp_list)
  
  temp_data = okt.morphs(trimmed_data["TITLE"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  trimmed_data["TITLE"].iloc[i] = " ".join(temp_list)

In [11]:
trimmed_data.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹를 주목했다 수비수가 볼을 터치하지 않았다고 봤다 멜레의 오른쪽 다리와 ...,2021-07-15
1,디 마리아 없다 유로X코파 베스트11 이탈리아만 7명,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까 지난달...,2021-07-15
2,슈퍼컴퓨터 예측 맨시티 우승맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다 영국 매체 스포츠 바이...,2021-07-15
3,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다 이재성은 20...,2021-07-15
4,홈킷과 딴판 바르사 팬들 NEW 어웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다 팬들은 만족스럽다는 반응이다 바르...,2021-07-15


# Model-Load
- 네이버의 하이퍼클로바
- 기반은 GPT-3
- 기본적으로 생성에 강점이 있는 모델이라 선정함

#Abstractive summarization

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader,random_split
from tqdm import tqdm

In [19]:
class CustomDataset(Dataset):
  def __init__(
      self,
      data,
      tokenizer,
      text_max_token_len: int = 512,
      summary_max_token_len : int = 128
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    data_row = self.data.iloc[index]

    text = data_row['CONTENT']

    text_encoding = tokenizer(
        text,
        max_length = self.text_max_token_len,
        padding = "max_length",
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt"
    )

    summary_encoding = tokenizer(
        text,
        max_length = self.summary_max_token_len,
        padding = "max_length",
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt"
    )

    labels = summary_encoding["input_ids"]

    return dict(
        text = text,
        summary = data_row["TITLE"],
        text_input_ids = text_encoding["input_ids"].flatten(),
        text_attention_mask = text_encoding["attention_mask"].flatten(),
        labels = labels.flatten(),
        labels_attention_mask = summary_encoding["attention_mask"].flatten()
    )


In [15]:
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

In [31]:
text = trimmed_data['CONTENT'].iloc[1]

text_encoding = tokenizer(
        text,
        max_length = 512,
        padding = "max_length",
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt",
    )
print(text_encoding['input_ids'])
tokenizer.decode(text_encoding['input_ids'].flatten())

tensor([[17412,  3248,   450, 19867,  9630,  5564,   392,  8232,  2010,   387,
          2456,  1523,  1540,  1602,  4008,  5464,  1157,   621,  8051,   533,
         19867,  9630,   402,  1203, 46339, 29460,   374,  7019,   378,  2012,
           474,  4560,   383,  5128,   376,   813,  4292,   837,  3469,   374,
          4560,   402,   887,   605, 11569,   378,   698,   605, 52426,   529,
           374,   812,   409,  3130,   376, 25534,   551,   708,  6135,   387,
         24188,   378,   410,   470,   418,   426,   529,   374,  6189,  8026,
           554,  6401,  1846,   383,  3668,  4443,  6898,  1390, 10288,   402,
          1431,  1521,  1549,  2693,  1492,   376,   529,   374,  2248,  3872,
           151,  4228,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,  

'유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까 지난달 시작된 유로와 코파 아메리카가 11일가 뽑혔다 매체는 비교적 이탈리아의 전력이 더 강하다고 봤다 이탈리아가 7명 아르헨티나는 4명뿐이었다 많은 팬이 수긍했지만 불만을 터뜨리는 이들도 있었다 대체로 브라질전 결승골의 주인공 앙헬 디 마리아가 포함됐어야 한다는 내용이었다 사진90MIN[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]

In [24]:
whole_dataset = CustomDataset(trimmed_data,tokenizer)

train_set_num = len(trimmed_data)//5*4
train_dataset , test_dataset = random_split(whole_dataset, [train_set_num,len(trimmed_data)-train_set_num])
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle=True)
test_dataloader =  DataLoader(test_dataset, batch_size = 32, shuffle=False)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

In [ ]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 수수께끼들을 풀 수 있게 되었다. 지능은 지금까지 인간의 두뇌 속에서만 존재하는 것으로 여겨져 왔지만, 최근에는 인간의 몸속에서도 발견되었


In [ ]:
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

In [ ]:
class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = output_file_name
        self.word2count, self.word2id, self.id2word = make_vocab(input_file,min_count)
        self.device = device
        self.emb_size = len(self.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        self.model = SkipGram(vocab_size = len(self.word2id), emb_dimension = emb_dimension, device = device)
        self.optimizer = torch.optim.SGD(self.model.parameters(),lr=lr) # torch.optim.SGD 클래스 사용

        # train() 함수에서 만든 임베딩 결과 파일들을 저장할 폴더 생성 (os.makedirs 사용)
        os.makedirs('./'+output_file_name,exist_ok=True)
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            optimizer = self.optimizer,
            num_warmup_steps=0,
            num_training_steps=self.iteration*len(train_dataloader)
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1
                pos_u, pos_v = batch
                # negative data 생성
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                
                # 데이터를 tensor화 & device 설정
                pos_u = Variable(torch.LongTensor(pos_u)).to(device)
                pos_v = Variable(torch.LongTensor(pos_v)).to(device)
                neg_v = Variable(torch.LongTensor(neg_v)).to(device)

                # model의 gradient 초기화
                self.model.init_embedding()
                # optimizer의 gradient 초기화
                self.optimizer.zero_grad()

                # forward
                loss = self.model.forward(pos_u,pos_v,neg_v)

                # loss 계산
                loss.backward()
                # optimizer 업데이트
                self.optimizer.step()
                # scheduler 업데이트
                self.scheduler.step()

                batch_loss += loss.item()
                total_loss += loss.item()
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")